---
title: "Loading data from Barb's asynchronous API using pybarb"
author: "Simon Raper"
toc: true
number-sections: true
highlight-style: pygments
format: 
  html:
    embed-resources: true
---


In this use case we will be using the asynchronous API to access the data. We will use the pybarb package to `pybarb` pull the data for a single day, a single station and a single panel and then reshape the data into viewer level data file (one row per viewer per programme).

## Set up the environment

In [1]:
import os
import re
import json
import pandas as pd

# Change the directory to the parent of the current directory
sub_dir = re.search('.+?\/barb_api', os.getcwd()).group(0)
os.chdir(sub_dir)
print(f"Working directory : {sub_dir}")

import pybarb as pb
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

Working directory : /Users/simon_business/Documents/code_repos/barb_api


## Connect to the API

In [2]:
# Read in Barb API credentials
with open("/Users/simon_business/Documents/disposable/clients/BARB/as_creds.json") as file:
    creds = json.load(file)

# Create a BarbAPI object and connect
api_root = "https://uat.barb-api.co.uk/api/v1/"
barb_api = pb.BarbAPI(creds, api_root)
barb_api.connect()


## Query the Asynchronous API

We would like one day's worth of viewing data (2023-05-28) for BBC1 East viewing station and the BBC East Region panel.

First we need to look up the viewing station and panels to check we have the correct query parameters

In [3]:
barb_api.list_panels("BBC")

['BBC Network',
 'BBC East Region',
 'BBC West Region',
 'BBC South West Region',
 'BBC South Region',
 'BBC Yorkshire & Lincolnshire',
 'BBC North East & Cumbria',
 'BBC North West Region',
 'BBC Scotland Region',
 'BBC Ulster Region',
 'BBC Wales Region',
 'BBC Midlands West',
 'BBC Midlands East',
 'BBC London',
 'BBC South East']

In [4]:
barb_api.list_viewing_stations("BBC1")

['BBC1 Midlands West',
 'BBC1 East',
 'BBC1 West',
 'BBC1 South West',
 'BBC1 South',
 'BBC1 Yorks/Lincs',
 'BBC1 North East/Cumbria',
 'BBC1 North West',
 'BBC1 Scotland',
 'BBC1 Wales',
 'BBC1 Northern Ireland',
 'BBC1 Midlands East',
 'BBC1 London',
 'BBC1 South East']

Next we need to request the data sets from the asynchronous API. We will use the `pybarb` package to do this.

In [5]:
barb_api.viewing(min_session_date="2023-06-06", max_session_date="2023-06-06", 
                 viewing_station="BBC1 East", panel="BBC East Region")

Job successfully started. The job id is 5bae7177-9165-4ed0-987a-1da618c736d3


Once the job has started we can use the ping_job_status method to check the status of the job. It will check the status every 60 seconds until the job is complete.

In [6]:
barb_api.ping_job_status()

Job not ready yet. Sleeping for 60 seconds.
Job not ready yet. Sleeping for 60 seconds.
Job complete. 1 files are ready for download.


Now we can download the data using the `get_asynch_files` method.

In [7]:
viewing_results_set = barb_api.get_asynch_files()

In its raw form  we can see many of the cells contain json. We will need to unpack this.

In [8]:
viewing_results_set.api_response_data

,STANDARD_DATE_OF_ACTIVITY,SESSION_START,SESSION_END,HOUSEHOLD,DEVICE,PANEL_VIEWERS,GUEST_VIEWERS,PROGRAMMES_VIEWED,SPOTS_VIEWED,VIEWING_STATION,PANEL,PLATFORM,ACTIVITY_TYPE,CONTENT_ASSET_ITEM_OFFSET,PLAYBACK_TYPE,SKY_ULTRA_HD,START_OF_RECORDING,TARGETED_PROMOTION,VOD_INDICATOR,VOD_PROVIDER
0,2023-06-06,{'barb_polling_datetime': '2023-06-06 22:12:00...,{'barb_polling_datetime': '2023-06-06 22:17:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1973-02-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",online via tv (& peripherals),live viewing (excl targeted advertising),0,unknown,False,{},False,not on-demand,"{'vod_provider': 'unknown', 'vod_service': 'un..."
1,2023-06-06,{'barb_polling_datetime': '2023-06-06 20:06:00...,{'barb_polling_datetime': '2023-06-06 20:07:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1999-11-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",digital terrestrial,live viewing (excl targeted advertising),0,unknown,False,{},False,not on-demand,"{'vod_provider': 'unknown', 'vod_service': 'un..."
2,2023-06-06,{'barb_polling_datetime': '2023-06-06 22:00:00...,{'barb_polling_datetime': '2023-06-06 22:32:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1963-03-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",digital terrestrial,live viewing (excl targeted advertising),0,unknown,False,{},False,not on-demand,"{'vod_provider': 'unknown', 'vod_service': 'un..."
3,2023-06-06,{'barb_polling_datetime': '2023-06-06 18:36:00...,{'barb_polling_datetime': '2023-06-06 20:06:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1983-06-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",digital terrestrial,vosdal (viewing on same day as live) (excl tar...,0,pvr device,False,{'barb_polling_datetime': '2023-06-06 18:02:00...,False,not on-demand,"{'vod_provider': 'unknown', 'vod_service': 'un..."
4,2023-06-06,{'barb_polling_datetime': '2023-06-06 22:36:00...,{'barb_polling_datetime': '2023-06-06 22:55:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1976-03-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",online via tv (& peripherals),timeshift (8-28 days) (excl targeted advertising),0,other device,False,{'barb_polling_datetime': '2023-05-26 21:12:00...,False,on-demand,"{'vod_provider': 'bbc iplayer', 'vod_service':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2023-06-06,{'barb_polling_datetime': '2023-06-06 18:24:00...,{'barb_polling_datetime': '2023-06-06 18:39:00...,{'bbc_itv_segment': 'bbc east / east of englan...,"{'date_valid_for': '2023-06-06', 'device_numbe...","[{'date_of_birth': '1950-04-01', 'dependency_o...",{},"[{'broadcaster_premier': True, 'broadcaster_tr...",[],"{'viewing_station_code': 32, 'viewing_station_...","{'is_macro_region': False, 'panel_code': 503, ...",digital terrestrial,live viewing (excl targeted advertising),0,unknown,False,{},False,not on-demand,"{'vod_provider': 'unknown', 'vod_service': 'un..."


Finally we reshape the data so that we have one row per viewer per programme.

In [9]:
df = viewing_results_set.to_dataframe(unpack=["viewers", "programmes"])
df.head()

,programme_start_datetime,programme_name,date_of_birth,dependency_of_children,disability,ethnic_origin,gaelic_language,household_status,life_stage,marital_status,person_membership_status,person_number,sex_code,terminal_age_of_education,welsh_language,working_status,bbc_itv_segment,bbc_region_code,bbc_sub_reporting_region,broadband,date_valid_for,demographic_cell,household_number,language_spoken_at_home,mosaic_classification_2014,number_of_computers,number_of_dvd_players_not_recorders,number_of_dvd_recorders,number_of_dvds,number_of_other_pvrs,number_of_people,number_of_pvrs,number_of_sky_plus_pvrs,number_of_tv_sets,number_of_vcrs,panel_membership_status,presence_of_children,replication_factor,social_class,welsh_speaking_home,device_number,device_on_panel,device_type
0,2023-06-06 22:00:00,BBC News at Ten: Series 2023,1973-02-01,unclassified,no,white british,not gaelic speaking/not in scotland,both houseperson and head of household,single no children on own or with friends aged...,single / divorced / separated,person on panel,1,female,16-18 years,not welsh speaking/not in wales,working 8-29 hours per week,bbc east / east of england,bbc east region,bbc east (east),True,2023-06-06,post family,68053,undefined,village retirement,999,2,0,2,1,1,1,0,4,0,home on panel (valid reporter),no children,6,D,non welsh speaking,1,True,tv
4,2023-06-06 20:00:03,"Waterloo Road: Series 12, Episode 4",1999-11-01,unclassified,no,white british,not gaelic speaking/not in scotland,neither houseperson nor head of household,single no children with parents aged 16-34,single / divorced / separated,person on panel,4,female,19 + years,not welsh speaking/not in wales,working 30+ hours per week,bbc east / east of england,bbc east region,bbc east (west),True,2023-06-06,post family,18630,undefined,village retirement,999,1,0,1,2,3,2,0,2,0,home on panel (valid reporter),no children,28,AB,non welsh speaking,1,True,tv
5,2023-06-06 20:00:03,"Waterloo Road: Series 12, Episode 4",1963-03-01,unclassified,no,white british,not gaelic speaking/not in scotland,both houseperson and head of household,either children 16+ none 0-15 aged 35+,married / living as married,person on panel,1,male,19 + years,not welsh speaking/not in wales,working 30+ hours per week,bbc east / east of england,bbc east region,bbc east (west),True,2023-06-06,post family,18630,undefined,village retirement,999,1,0,1,2,3,2,0,2,0,home on panel (valid reporter),no children,28,AB,non welsh speaking,1,True,tv
6,2023-06-06 20:00:03,"Waterloo Road: Series 12, Episode 4",1967-09-01,unclassified,no,white british,not gaelic speaking/not in scotland,neither houseperson nor head of household,either children 16+ none 0-15 aged 35+,married / living as married,person on panel,2,female,19 + years,not welsh speaking/not in wales,working 30+ hours per week,bbc east / east of england,bbc east region,bbc east (west),True,2023-06-06,post family,18630,undefined,village retirement,999,1,0,1,2,3,2,0,2,0,home on panel (valid reporter),no children,28,AB,non welsh speaking,1,True,tv
16,2023-06-06 22:00:00,BBC News at Ten: Series 2023,1963-03-01,unclassified,no,white british,not gaelic speaking/not in scotland,both houseperson and head of household,either children 16+ none 0-15 aged 35+,married / living as married,person on panel,1,male,19 + years,not welsh speaking/not in wales,working 30+ hours per week,bbc east / east of england,bbc east region,bbc east (west),True,2023-06-06,post family,18630,undefined,village retirement,999,1,0,1,2,3,2,0,2,0,home on panel (valid reporter),no children,28,AB,non welsh speaking,1,True,tv
